In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45180
2,Huelva,Confirmados PDIA 14 días,371
3,Huelva,Tasa PDIA 14 días,"72,29573045969173"
4,Huelva,Confirmados PDIA 7 días,184
5,Huelva,Tasa PDIA 7 dias,"35,85556443283902"
6,Huelva,Total Confirmados,45382
7,Huelva,Curados,43540
8,Huelva,Fallecidos,412


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45180.0


/tmp/ipykernel_3559/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12495.0


/tmp/ipykernel_3559/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3559/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3559/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3559/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3785 personas en los últimos 7 días 

Un positivo PCR cada 1844 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45180.0,184.0,371.0,513170.0,35.855564,72.295730,77.0
Huelva-Costa,26743.0,106.0,241.0,289548.0,36.608783,83.233177,38.0
Condado-Campiña,14023.0,69.0,101.0,156231.0,44.165371,64.647861,34.0
Huelva (capital),12495.0,38.0,78.0,143837.0,26.418793,54.228050,19.0
Bollullos Par del Condado,1273.0,14.0,14.0,14387.0,97.310072,97.310072,12.0
Ayamonte,2057.0,21.0,42.0,21104.0,99.507202,199.014405,8.0
Gibraleón,930.0,7.0,7.0,12737.0,54.957996,54.957996,6.0
Cartaya,1955.0,15.0,49.0,20083.0,74.690036,243.987452,5.0
Moguer,1948.0,13.0,20.0,21867.0,59.450313,91.462020,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Higuera de la Sierra,69.0,4.0,5.0,1291.0,309.837335,387.296669,2.0
Paterna del Campo,206.0,11.0,13.0,3457.0,318.194967,376.048597,1.0
Santa Bárbara de Casa,46.0,1.0,3.0,1043.0,95.877277,287.631831,0.0
Cartaya,1955.0,15.0,49.0,20083.0,74.690036,243.987452,5.0
Ayamonte,2057.0,21.0,42.0,21104.0,99.507202,199.014405,8.0
Isla Cristina,3109.0,9.0,29.0,21393.0,42.069836,135.558360,0.0
Palos de la Frontera,1014.0,4.0,13.0,11742.0,34.065747,110.713677,1.0
Bollullos Par del Condado,1273.0,14.0,14.0,14387.0,97.310072,97.310072,12.0
Lepe,2997.0,13.0,26.0,27880.0,46.628407,93.256815,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cartaya,1955.0,15.0,49.0,20083.0,74.690036,243.987452,5.0,0.306122
Palos de la Frontera,1014.0,4.0,13.0,11742.0,34.065747,110.713677,1.0,0.307692
Isla Cristina,3109.0,9.0,29.0,21393.0,42.069836,135.558360,0.0,0.310345
Santa Bárbara de Casa,46.0,1.0,3.0,1043.0,95.877277,287.631831,0.0,0.333333
Sierra de Huelva-Andévalo Central,3990.0,9.0,26.0,67391.0,13.354899,38.580819,3.0,0.346154
Huelva-Costa,26743.0,106.0,241.0,289548.0,36.608783,83.233177,38.0,0.439834
Huelva (capital),12495.0,38.0,78.0,143837.0,26.418793,54.228050,19.0,0.487179
Huelva,45180.0,184.0,371.0,513170.0,35.855564,72.295730,77.0,0.495957
Ayamonte,2057.0,21.0,42.0,21104.0,99.507202,199.014405,8.0,0.500000
